<img src = "Jup_nb_imgs/CVI.jpg" width = "500" height = "500">

#                        Computer Vision and Intelligence Group, CFI- IIT Madras

## Solutions to exercises and Mini Project
### (Session 2: Colour Spaces, Edge Detection and Road Lane Segmentation Mini Project)


In [2]:
#EXERCISE: GETTING STARTED WITH IMAGES- READING, DISPLAYING AND WRITING AN IMAGE

#Using the functions described above, write a program
#to read the image 'sample.jpg' in BGR and Grayscale separately
#display them and save the Grayscale image as 'sample_BW.jpg'
#NOTE: Path of sample.jpg is "sample_imgs/sample.jpg"; write sample_BW.jpg to the path "sample_imgs/sample_BW.jpg" 
import numpy as np
import cv2

img1 = cv2.imread('sample_imgs/sample.jpg', 1)       #READING AN IMAGE, in Colour 
img2 = cv2.imread('sample_imgs/sample.jpg', 0)       #READING the image in Grayscale
cv2.imshow('stonehenge', img1)           #DISPLAYING AN IMAGE
cv2.imshow('stonehenge_BW', img2)

cv2.imwrite('sample_imgs/sample_BW.jpg', img2)       #Saving the Black and White image to the specified path

cv2.waitKey(0)                           #Waits for a key press indefinitely (when argument = 0) to proceed to next line
cv2.destroyAllWindows()

#We have written a program in the next cell, that you may refer in case you are stuck

In [2]:
#EXERCISE: CHANGING COLOUR SPACES

#Using the functions described above, write a program
#to read the image 'hat_lady.png' in BGR 
#convert them to GRAY and HSV, 
#use numpy slicing to store hue, saturation and value channels separately in variables
#display all the images (except the input) in separate windows
#NOTE: Path of hat_lady.png is "sample_imgs/hat_lady.png"

img = cv2.imread('sample_imgs/hat_lady.png')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#Slicing the HSV image into components
hue = img_hsv[:, :, 0]
sat = img_hsv[:, :, 1]
val = img_hsv[:, :, 2]

cv2.imshow('Gray', img_gray)
cv2.imshow('HSV', img_hsv)
#Displaying components of the image
cv2.imshow('Hue', hue)
cv2.imshow('Saturation', sat)
cv2.imshow('Value', val)
cv2.waitKey(0)
cv2.destroyAllWindows()

#We have written a program in the next cell, that you may refer in case you are stuck

In [3]:
#2D CONVOLUTION OPERATION

#Write a program that performs a 2D convolution on the image 'hat_lady.png' with a 3x3 kernel
#Create a random kernel with element values of your choice
#Remember that if the kernel weights are too high or too low, the pixels get thresholded to 255 or 0 respectively

#conv_kernel = np.array([[0.5, -0.5, 0.5],[0, 0.5, 0],[0.5, -0.5, 0.5]])
conv_kernel = np.array([[0.111, 0.111, 0.111],[0.111, 0.111, 0.111],[0.111, 0.111, 0.111]])
#conv_kernel = np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]])
dst_img = np.zeros(np.shape(img))
dst_img = cv2.filter2D(img, -1, conv_kernel)
cv2.imshow('org', img)
cv2.imshow('Convolved Image', dst_img)
cv2.waitKey(0)
cv2.destroyWindow('Convolved Image')

#We have written a program in the next cell, that you may refer in case you are stuck

In [4]:
#EDGE DETECTION

#Write a Program that performs Gaussian Bluriing and Canny edge Detection on 'hat_lady.png'
#Display the Canny Output

img_blur = cv2.GaussianBlur(img, (3, 3), 1)
canny_output = cv2.Canny(img_blur, 80, 200)   #Try varying the thresholds
cv2.imshow('Canny Output', canny_output)
cv2.waitKey(0)
cv2.destroyWindow('Canny Output')

#We have written a program in the next cell, that you may refer in case you are stuck

### Structure for the Lane Detector 

We may define the following functions in order to make the program more modular:

**img2edge(img):** 

A function which takes the input image (img), uses a mask to pick yellow and white pixels, does Gaussian Blurring and Canny Edge Detection. 

Returns canny_output (a contour of edges)
             
**roi_select(img, canny_output):** 

Reads in the canny_output, returns canny_roi: canny_output with irrelevant regions of the image reset to 0 (blacked out). The RoI is defined using dimensions of img.

**draw_lines(canny_roi, rho_acc, theta_acc, thresh, minLL, maxLG):** 

Inputs canny_roi, performs Probabilistic Hough Transform on it. Draws lines on a blank image using the values returned by HoughLinesP() and the openCV function cv2.line()

Parameters for HoughLinesP() are passed in as parameters to draw_lines()

Returns img_lines: image of lines against a black background
                         
**add_weighted(img, img_lines):**

Computes a weighted sum of the original image and the image  of lines. 

Returns lane_img: image with lanes

In [3]:
# MINI-PROJECT: LANE DETECTION

#Input for Program: Image of a road with clearly labelled lanes
#Expected Output: Lanes highlighted with white superimposed on the original image
#Store the input images in the folder 'LD_test_imgs' and save the output images in 'LD_return_imgs'
#You may check the following functions out:
# cv2.inRange()
# cv2.bitwise_or() and cv2.bitwise_and
# cv2.fillPoly()
# cv2.line()
# cv2.addWeighted()
#In addition, you may consider the structure of the program (functions that one may define) we discussed in the previous cell

def img2edge(img):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #defining ranges for 'yellow', pixels within this range will be picked
    lower_yellow = np.array([20, 100, 100], dtype = "uint8")    
    upper_yellow = np.array([30, 255, 255], dtype="uint8")
    #
    #cv2.inRange(): Picks pixels from the image that are in the specified range
    mask_y = cv2.inRange(hsv_image, lower_yellow, upper_yellow)     
    mask_w = cv2.inRange(gray_image, 216, 255)
    #Computing Bitwise OR, combining both the white and yellow pixels
    mask_yw = cv2.bitwise_or(mask_w, mask_y)
    #Computing Bitwise AND of mask_yw with gray_img: pixels that were yellow or white will have the same intensity as the
    #original grayscale image, the other pixels will be removed
    mask_yw_image = cv2.bitwise_and(gray_image, mask_yw)
    #
    img_blur = cv2.GaussianBlur(mask_yw_image, (3, 3), 0)
    img_canny = cv2.Canny(img_blur, 70, 200)
    return img_canny
    
def roi_select(img, canny):
    #Defining vertices of the region of interest
    imshape = img.shape
    lower_left = [0,imshape[0]]
    lower_right = [imshape[1], imshape[0]]
    top_left = [0, imshape[0]/(3.2)]
    top_right = [imshape[1], imshape[0]/(3.2)]
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]
    #
    mask = np.zeros_like(canny)      #creates a numpy array of the same dimensions as img
    fill_color = 255               #parameter for cv2.fillPoly function
    cv2.fillPoly(mask, vertices, fill_color) #pixels within 'vertices' in 'mask' will be made WHITE while all other pixels will be BLACK
    return cv2.bitwise_and(canny, mask)

def draw_lines(canny_roi, rho_acc, theta_acc, thresh, minLL, maxLG):
    lines = cv2.HoughLinesP(canny_roi, rho_acc, theta_acc, thresh, np.array([]), minLL, maxLG)
    line_img = np.zeros((canny_roi.shape[0], canny_roi.shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), [255, 255, 255], 2)  
    return line_img

def add_weighted(img, line_img):
    return cv2.addWeighted(img, 0.8, line_img, 1, 0)

img = cv2.imread('LD_test_imgs/test_img01.jpeg')
edge_img = img2edge(img)
roi_img = roi_select(img, edge_img)
hough_img = draw_lines(roi_img, 2, np.pi/180, 50, 50, 100)   
#Change the parameters thresh, minLL, maxLG to get more accurate lines
#Also test this on multiple images downloaded from the internet
lane_img = add_weighted(img, hough_img)
cv2.imshow('edge', edge_img)
cv2.imshow('roi', roi_img)
cv2.imshow('Hough lines', hough_img)
cv2.imshow('Image with lanes', lane_img)
cv2.imwrite('LD_return_imgs/test_img01.jpeg', lane_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#We have written a program in the next cell, that you may refer in case you are stuck